# Исследование надежности заемщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Цель исследования** - проверить четыре гипотезы:
1. Возврат кредита в срок зависит от количеством детей.
2. Возврат кредита в срок зависит от семейного положения. 
3. Возврат кредита в срок зависит от уровня дохода.
4. Разные цели кредита влияют на его возврат в срок.

**Ход исследования**

Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.

## Обзор данных

Выведем на экран первые пятнадцать строк таблицы:

In [287]:
import pandas as pd # импорт библиотеки pandas

In [288]:
df = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в df
df.head(15) # получение первых 15 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Получим общую информацию о таблице.

In [289]:
df.info() # получение общей информации о данных в таблице df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


В таблице двенадцать столбцов. Типы данных в столбцах разные — float64, int64, object.

Согласно документации к данным:

* `children` — количество детей в семье;
* `days_employed` — общий трудовой стаж в днях;
* `dob_years` — возраст клиента в годах;
* `education` — уровень образования клиента;
* `education_id` — идентификатор уровня образования;
* `family_status` — семейное положение;
* `family_status_id` — идентификатор семейного положения;
* `gender` — пол клиента;
* `income_type` — тип занятости;
* `debt` — имел ли задолженность по возврату кредитов;
* `total_income` — ежемесячный доход;
* `purpose` — цель получения кредита.

**Выводы**

В каждой строке таблицы — данные о заёмщиках. Часть колонок описывает финансовые данные: наличие задолженности, ежемесячный доход и цель получения кредита. Остальные данные рассказывают о клиенте: наличие детей, трудовой стаж, возраст, образование, семейное положение, пол и тип занятости. 

Предварительно можно утверждать, что данных достаточно для проверки гипотез. Но встречаются некорректные данные (некоторые значения трудового стажа отрицательные, сведения об образовании записаны с нарушением стиля) и имеются пропуски в данных.

Необходимо устранить проблемы в данных.

## Предобработка данных

### Заполнение пропусков

Для начала посчитаем количество пропущенных в таблице значений.

In [290]:
df.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения обнаружены в столбцах `days_employed` (общий трудовой стаж в днях) и `total_income` (ежемесячный доход). Причина появления пропусков в данных скорее всего состоит в том, что клиенты не захотели указать эти данные по каким-либо причинам. 

In [291]:
print('Доля пропусков в данных составляет:', round(2174 / 21525 * 100, 2), '%')

Доля пропусков в данных составляет: 10.1 %


Так как остальные значения в столбцах с пропусками представлены в широком диапазоне и всегда есть вероятность присутствия слишком больших значений, влияющих на распределение данных в выборке, заменим пропуски медианным значением.

Примем во внимание, что и общий трудовой стаж в днях и ежемесячный доход напрямую зависит от типа занятости. Поэтому заменим пропуски в таблице медианными значениями по группе `тип занятости`.

In [292]:
for income in df['income_type'].unique(): # формирование цикла по уникальным значениям типа занятости
    median_1 = df.loc[df['income_type'] == income, 'days_employed'].median() # расчет медианы для столбца 'days_employed'
    median_2 = df.loc[df['income_type'] == income, 'total_income'].median() # расчет медианы для столбца 'total_income'
    df.loc[(df['days_employed'].isna()) & (df['income_type'] == income), 'days_employed'] = median_1 # замена пропусков
    df.loc[(df['total_income'].isna()) & (df['income_type'] == income), 'total_income'] = median_2 # замена пропусков


Убедимся, что в таблице не осталось пропусков. Для этого ещё раз посчитаем пропущенные значения.

In [293]:
df.isna().sum() # подсчёт пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

И еще раз выведем на экран первые пятнадцать строк таблицы:

In [294]:
df.head(15) # получение первых 15 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Проверка данных на аномалии и исправления

Для лучшей читаемости таблицы округлим значения в столбце `total_income` до двух знаков после запятой. 

In [295]:
df['total_income'] = round(df['total_income'], 2) # округление значений до двух знаков после запятой
df.head() # проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу


Проверим на аномалии столбец `children`.

In [296]:
df['children'].value_counts() # проверка аномалий в столбце 'children'

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Здесь определяются две проблемы. Отрицательное значение, и слишком большое значение. Обе ошибки с большой долей вероятностью вызваны банальными "описками". Для отрицательного значения (-1), мы перезапишем данные этого столбца, используя функцию с определением модуля числа. Для слишком большого значения (20), произведем замену 20 на 2, так как и среднее значение, и медианное, списка чисел от 0 до 5 одинаковое и равно 2.5.

In [297]:
df['children'] = abs(df['children']) # замена отрицательных значений в столбце 'children' на модуль
df['children'] = df['children'].replace(20, 2) # замена значения '20' на '2'
df['children'].value_counts() # проверка

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Проверим на аномалии столбец `dob_years`.

In [298]:
df['dob_years'].value_counts() # проверка аномалий в столбце 'dob_years'

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Видно, что у 101 клиента в графе возраст стоит 0, это может быть как нежеланием клиентов указывать свой возраст, так и технической ошибкой. Заменим нули медианным значением.

In [299]:
df['dob_years'] = df['dob_years'].replace(0, df['dob_years'].median()) # замена нулевых значений медианным
df['dob_years'].value_counts() # проверка

42    698
35    617
40    609
41    607
34    603
38    598
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Обработаем аномалии в данных по трудовому стажу, у части клиентов они отображаются в отрицательном диапазоне. Это может быть связано с некорректным наполнением таблицы либо вручную, либо неполадками при импорте с других источников.

In [300]:
df['days_employed'] = abs(df['days_employed']) # замена отрицательных значений в столбце 'days_employed' на модуль

Также были замечены аномально высокие данные по трудовому стажу, необходимо их обработать. Для начала определимся, с какого значения трудовой стаж будет аномально большим.

Согласно статистике в РФ за последние 5 лет было в среднем 247 рабочих дней в году, учитывая положенный трудовым законодательством ежегодный отпуск в 28 дней (за год), примем 220 рабочих дней в году. Также необходимо учесть ограничение трудового стажа в виде пенсионного возраста. Рассмотрим идеальный вариант: женщина, которая начала трудовую деятельность в 18 лет и работала непрерывно до 60 лет. Учитывая, что у мужчин выход на пенсию позже, а также тот факт, что крайне мало людей работает непрерывно и начинают не в 18 лет, можно получить адекватную границу максимально возможного трудового стажа.

In [301]:
print('Максимально возможный трудовой стаж составляет:', (60 - 18) * 220, 'дней')

Максимально возможный трудовой стаж составляет: 9240 дней


Заменим аномально большие значения трудового стажа по следующей логике: рассчитаем медианное значение трудового стажа для каждого значения возраста в выборке клиентов, у которых значение трудового стажа меньше или равно 9240 дней и заменим ими аномально большой трудовой стаж. 

In [302]:
zf = df[df['days_employed'] <= 9240] # формирование нового датафрейма с выборкой клиентов, у которых стаж меньше или равен 9240 дням
for days in df['dob_years'].unique(): # формирование цикла по уникальным значениям возраста
    median = zf.loc[df['dob_years'] == days, 'days_employed'].median() # расчет медианы для столбца 'days_employed' в выборке
    df.loc[(df['days_employed'] > 9240) & (df['dob_years'] == days), 'days_employed'] = median # замена аномально большого стажа

Проверим, выведя на экран первые десять строк таблицы:

In [303]:
df.head(10) # проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,1816.843492,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.93,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.83,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,покупка жилья для семьи


### Изменение типов данных

Заменим тип данных в столбце `total_income` с вещественного на целочисленный, это избавит таблицу от визуальной перегруженности, а точность расчетов не пострадает.

In [304]:
df['days_employed'] = df['days_employed'].astype('int') # замена вещественного типа данных в столбце 'days_employed'
# на целочисленный

Проверим, выведя на экран первые пять строк таблицы:

In [305]:
df.head() # получение первых 5 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,1816,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу


### Удаление дубликатов

Для начала определим, есть ли в таблице явные дубликаты. Используем метод `duplicated()` в сочетании с `sum()`, так как нам необходимо увидеть в каких столбцах присутсвуют повторы в данных.

In [306]:
df.duplicated().sum() # подсчёт явных дубликатов

55

Выведем на экран первые несколько строк таблицы с повторяющимися строками для ознакомления.

In [307]:
df[df.duplicated()].head() # получение первых 5 строк таблицы с повторяющимися строками

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,1574,41,среднее,1,женат / замужем,0,F,сотрудник,0,142594.40,покупка жилья для семьи
4182,1,1574,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,142594.40,свадьба
4851,0,2090,60,среднее,1,гражданский брак,1,F,пенсионер,0,118514.49,свадьба
5557,0,1860,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514.49,сыграть свадьбу
7808,0,1574,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514.49,на проведение свадьбы


Анализ таблицы с повторяющимися строками позволяет сделать вывод, что совпадения имеются в основном по количеству дней трудового стажа и величине ежемесячного дохода. Такое дублирование данных возможно как из-за ручного заполнения, так и проблем при импорте данных со сторонних ресурсов. Удалим повторяющиеся строки c изменением индексации строк.

In [308]:
df = df.drop_duplicates().reset_index(drop=True) # удаление повторяющихся строк (с удалением старых индексов и формированием новых)

Убедимся, что избавились от повторяющихся строк в таблице.

In [309]:
df.duplicated().sum() # проверка на отсутствие повторяющихся строк

0

В столбце `education` присутствуют данные, записанные с использованием букв разного регистра. Для начала проанализируем этот столбец с помощью метода `value_counts()`.

In [310]:
df['education'].value_counts() # подсчёт неявных дубликатов

среднее                13704
высшее                  4710
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   273
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Исправим различное написание данных об образовании, приведя всё к одному регистру. Такие ошибки в написании свойственны при внесении данных ручным способом разными людьми.

In [311]:
df['education'] = df['education'].str.lower() # приведение всех символов в столбце 'education' к нижнему регистру
df['education'].value_counts() # проверка

среднее                15187
высшее                  5251
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Дополнительно проверим датасет, выведя на экран первые десять строк.

In [312]:
df.head(10) # проверка

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,1816,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.57,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.97,операции с жильем
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.93,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.83,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.94,покупка жилья для семьи


Проверим на неявные дубликаты остальные столбцы.

In [313]:
df['family_status'].value_counts() # подсчёт неявных дубликатов в столбце 'family_status'

женат / замужем          12344
гражданский брак          4162
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Здесь можно исправить написание категории `Не женат / не замужем` строчными буквами, в общем стиле со остальными данными в этом столбце

In [314]:
df['family_status'] = df['family_status'].replace('Не женат / не замужем','не женат / не замужем') # исправление альтернативного
# значения категории в столбце 'family_status'
df['family_status'].value_counts() # проверка

женат / замужем          12344
гражданский брак          4162
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [315]:
df['gender'].value_counts() # подсчёт неявных дубликатов в столбце 'gender'

F      14188
M       7281
XNA        1
Name: gender, dtype: int64

Выведем всю строку со значением `XNA`.

In [316]:
df[df['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10690,0,2358,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.16,покупка недвижимости


Ситуацию это не прояснило, примем решение заменить `'XNA'` на `'M'`, так как это значение встречается всего один раз на 21525 значений и это не повлияет на точность расчетов в дальнейшем.

In [317]:
df['gender'] = df['gender'].replace('XNA', 'M') # замена 'XNA' на 'M'
df.loc[10690, 'gender'] # проверка

'M'

In [318]:
df['income_type'].value_counts() # подсчёт неявных дубликатов в столбце 'income_type'

сотрудник          11090
компаньон           5080
пенсионер           3837
госслужащий         1457
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Здесь данные в порядке и не требуют каких-либо корректировок.

In [319]:
df['purpose'].value_counts() # подсчёт неявных дубликатов в столбце 'purpose'

свадьба                                   793
на проведение свадьбы                     772
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

На мой взгляд, здесь также пристутствуют неявные дубликаты. Схожие цели выдачи кредита записаны по-разному. Можно уменьшить количество целей выдачи кредита, объединив похожие, что я первоначально и сделал. Но категоризация этого типа данных пристутствует дальше в проекте, поэтому на данном этапе ничего делать с этими данными не нужно.

### Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

Создадим два новых датафрейма, в которых:
* каждому уникальному значению из `education` соответствует уникальное значение `education_id` — в первом;
* каждому уникальному значению из `family_status` соответствует уникальное значение `family_status_id` — во втором.

In [320]:
education_log = df[['education','education_id']] # создание датафрейма, в котором каждому уникальному значению из 'education'
# соответствует уникальное значение 'education_id'
family_status_log = df[['family_status','family_status_id']] # создание датафрейма, в котором каждому уникальному значению  
# из 'family_status' соответствует уникальное значение 'family_status_id'

Посмотрим, что получилось, выведя на экран первые пять строк каждого датафрейма.

In [321]:
education_log.head() # получение первых 5 строк датафрема 'education_log'

,education,education_id
0,высшее,0
1,среднее,1
2,среднее,1
3,среднее,1
4,среднее,1


In [322]:
family_status_log.head() # получение первых 5 строк датафрема 'family_status_log'

,family_status,family_status_id
0,женат / замужем,0
1,женат / замужем,0
2,женат / замужем,0
3,женат / замужем,0
4,гражданский брак,1


Удалим из полученных датафреймов дубликаты.

In [323]:
education_log = education_log.drop_duplicates().reset_index(drop=True) # удаление дубликатов из датафрема 'education_log'
family_status_log = family_status_log.drop_duplicates().reset_index(drop=True) # удаление дубликатов из датафрема 'family_status_log'
display(education_log.head()) # проверка
family_status_log.head() # проверка

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


Удалим из исходного датафрейма столбцы `education` и `family_status`, оставив только их идентификаторы: `education_id` и `family_status_id`.

In [324]:
df = df.drop(columns=['education', 'family_status']) # удаление столбцов 'education' и 'family_status'
df.head() # проверка

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875.64,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885.95,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,1816,53,1,1,F,пенсионер,0,158616.08,сыграть свадьбу


### Категоризация дохода

Создадим столбец `total_income_category`, в котором присвоим категории в зависимости от размера доходов клиента:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

Для начала составим функцию.

In [325]:
def income_group(income): # возвращает соответствующую уровню дохода группу
    if income <= 30000:
        return 'E'
    if income <= 50000:
        return 'D'
    if income <= 200000:
        return 'C'
    if income <= 1000000:
        return 'B'
    return 'A'

Создадим новый столбец `total_income_category`.

In [326]:
df['total_income_category'] = df['total_income'].apply(income_group) # создание нового столбца
df.head() # проверка

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875.64,покупка жилья,B
1,1,4024,36,1,0,F,сотрудник,0,112080.01,приобретение автомобиля,C
2,0,5623,33,1,0,M,сотрудник,0,145885.95,покупка жилья,C
3,3,4124,32,1,0,M,сотрудник,0,267628.55,дополнительное образование,B
4,0,1816,53,1,1,F,пенсионер,0,158616.08,сыграть свадьбу,C


### Категоризация целей кредита

Создадим столбец `purpose_category`, в котором присвоим категории в зависимости от целей получения кредита:
* 'операции с автомобилем';
* 'операции с недвижимостью';
* 'проведение свадьбы';
* 'получение образования'.

Для начала составим функцию.

In [327]:
def purpose_group(purpose): # возвращает соответствующую уровню дохода группу
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образовани' in purpose:
        return 'получение образования'
    if 'автомобил' in purpose: 
        return 'операции с автомобилем'
    return 'операции с недвижимостью'

Создадим новый столбец `purpose_category`.

In [328]:
df['purpose_category'] = df['purpose'].apply(purpose_group) # создание нового столбца
df.head() # проверка

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875.64,покупка жилья,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080.01,приобретение автомобиля,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885.95,покупка жилья,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628.55,дополнительное образование,B,получение образования
4,0,1816,53,1,1,F,пенсионер,0,158616.08,сыграть свадьбу,C,проведение свадьбы


Дополнительно проверим результат категоризации данных по целям кредита.

In [329]:
df['purpose_category'].value_counts() # проверка

операции с недвижимостью    10814
операции с автомобилем       4308
получение образования        4014
проведение свадьбы           2334
Name: purpose_category, dtype: int64

**Выводы**

Предобработка обнаружила несколько проблем в данных:

- нарушения в отображении данных (использование букв разного регистра, количество знаков после запятой, отрицательные значения, аномально большие значения),
- пропущенные значения,
- дубликаты — явные и неявные.

Исправив проблемы, упростили работу с таблицей, что в дальнейшем поможет сделать исследование более точным.

Также была проведена работа по категоризации данных и декомпозиции датафрейма, что поможет упростить проверку гипотез.

## Проверка гипотез

### Зависимость возврата кредита в срок от количества детей

Необходимо сопоставить информацию по количеству детей в семье и наличию задолженности по кредитам. Для этого сформируем сводную таблицу по следующему принципу - в первом столбце `children` у нас будет отображаться количество детей в семье, во втором столбце `children len` будет отображаться количество семей с текущим количеством детей, в третьем столбце `debt mean` будет отображаться доля семей у которых имеется задолженность при текущем количестве детей (арифметическое среднее по данным из столбца 'debt'), в четвертом столбце `debt sum` будет отображаться количество семей-должников при текущем количестве детей. 

In [330]:
import numpy as np # импортируем библиотеку numpy

In [331]:
df.pivot_table(index='children', aggfunc={'children': len, 'debt': [np.mean, np.sum]}) # составление сводной таблицы

children      debt        
              len      mean     sum
children                           
0           14106  0.075358  1063.0
1            4856  0.091639   445.0
2            2128  0.094925   202.0
3             330  0.081818    27.0
4              41  0.097561     4.0
5               9  0.000000     0.0

### Вывод по первой гипотезе

По результатам проверки первой гипотезы мы видим, что при максимальном количестве детей в семье (5) долги по кредитам отсутствуют. В группах с количеством детей от 1 до 4 процент семей, имеющих задолженность по кредитам варьируется от 8.18 % до 9.76 %. При отсутствии детей, процент семей, имеющих долги по кредитам равен 7.54 % от группы. Это позволяет сделать вывод, что семьи с количеством детей 0 и 5, как правило либо не имеют долгов, либо стараются их вовремя возвращать. Семьи с количеством детей от 1 до 4 могут допускать просрочки по кредитам.

### Зависимость возврата кредита в срок от семейного положения

Необходимо сопоставить информацию по семейному положению и наличию задолженности по кредитам. Для этого сформируем сводную таблицу по следующему принципу - в первом столбце `family_status_id` у нас будет отображаться семейное положение, во втором столбце `debt mean` будет отображаться доля клиентов у которых имеется задолженность при текущем семейном статусе (арифметическое среднее по данным из столбца 'debt'), в третьем столбце `debt sum` будет отображаться количество клиентов у которых присутствуют задолженности при текущем семейном статусе, в четвертом столбце `family_status_id len` будет отображаться количество клиентов с текущим семейным статусом. При этом следует помнить, что в предыдущих итерациях из оригинального датафрейма был удален столбец `family_status`.

In [332]:
df.pivot_table(index='family_status_id', aggfunc={'family_status_id': len, 'debt': [np.mean, np.sum]}) # составление сводной таблицы

debt        family_status_id
                      mean    sum              len
family_status_id                                  
0                 0.075421  931.0            12344
1                 0.093224  388.0             4162
2                 0.065693   63.0              959
3                 0.071130   85.0             1195
4                 0.097509  274.0             2810

In [333]:
family_status_log # шпаргалка

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


### Вывод по второй гипотезе

По результатам проверки второй гипотезы мы видим, что наибольшее количество долгов имеют не женатые/не замужние и клиенты, состоящие в гражданском браке, 9.75 % и 9.32 % соответственно. Женатые/замужние и клиенты, находящиеся в разводе имеют средние показатели 7.54 % и 7.11 %. Самые аккуратные к своим долгам вдовцы/вдовы, их всего 6.57 % от группы.

### Зависимость возврата кредита в срок от уровня дохода

Необходимо сопоставить информацию по уровню дохода и наличию задолженности по кредитам. Для этого сформируем сводную таблицу по следующему принципу - в первом столбце `total_income_category` у нас будет отображаться категория дохода клиента, во втором столбце `debt mean` будет отображаться доля клиентов у которых имеется задолженность при текущей категории дохода (арифметическое среднее по данным из столбца 'debt'), в третьем столбце `debt sum` будет отображаться количество клиентов у которых присутствуют задолженности при текущей категории дохода, в четвертом столбце `total_income_category len` будет отображаться количество клиентов с текущей категории дохода. При этом следует помнить, что в предыдущих итерациях производилась категоризация клиентов по уровню дохода.

In [334]:
df.pivot_table(index='total_income_category', aggfunc={'total_income_category': len, 'debt': [np.mean, np.sum]}) # составление сводной таблицы

debt         total_income_category
                           mean     sum                   len
total_income_category                                        
A                      0.080000     2.0                    25
B                      0.070607   356.0                  5042
C                      0.084836  1360.0                 16031
D                      0.060000    21.0                   350
E                      0.090909     2.0                    22

Шпаргалка по категориям дохода:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.

### Вывод по третьей гипотезе

По результатам проверки третьей гипотезы мы видим, что наибольшее количество просрочек допускают клиенты с минимальным уровнем дохода (0-30000), их 9.09 % от группы. Клиенты, имеющие максимальный (1000001 и выше) и средний уровень дохода (50001–200000) имеют средние показатели 8.48 % и 8.00 %. Клиенты, имеющие доход "выше среднего" (200001–1000000) и "ниже среднего" (30001–50000), самые аккуратные в плане возвратов кредита в срок, их показатели 7.06 % и 6.00 % соответственно.

### Влияние разных целей на возврат кредита в срок

Необходимо сопоставить информацию по целям кредита и наличию задолженности по кредитам. Для этого сводную таблицу по следующему принципу - в первом столбце `purpose_category` у нас будет отображаться цель кредита клиента, во втором столбце `debt mean` будет отображаться доля клиентов у которых имеется задолженность при текущей цели кредита (арифметическое среднее по данным из столбца 'debt'), в третьем столбце `debt sum` будет отображаться количество клиентов у которых присутствуют задолженности при текущей цели кредита, в четвертом столбце `purpose_category len` будет отображаться количество клиентов с текущей целью кредита. При этом следует помнить, что в предыдущих итерациях производилась категоризация клиентов по целям кредита.

In [335]:
df.pivot_table(index='purpose_category', aggfunc={'purpose_category': len, 'debt': [np.mean, np.sum]}) # составление сводной таблицы

debt        purpose_category
                              mean    sum              len
purpose_category                                          
операции с автомобилем    0.093547  403.0             4308
операции с недвижимостью  0.072314  782.0            10814
получение образования     0.092177  370.0             4014
проведение свадьбы        0.079692  186.0             2334

### Вывод по четвертой гипотезе

По результатам проверки четвертой гипотезы мы видим, что наибольшее количество просрочек допускают клиенты, которые планируют брать кредиты на операции с автомобилем и получение образования, их 9.35 % и 9.22 % от групп. Клиенты, планирующие брать кредит на операции с недвижимостью и проведение свадьбы стараются не допускать просрочек, их показатели 7.23 % и 7.97 % соответственно.

## Итоги исследования

Я проверил четыре гипотезы и установил:

1. Количество детей влияет на срок возврата кредита.
2. Семейное положение влияет на срок возврата кредита. 
3. Уровень дохода влияет на срок возврата кредита. 
4. Цель кредита влияет на срок возврата кредита. 

Все гипотезы подтвердились.
Однако в ходе проверки выяснилось, что не стоит делать однозначные выводы по степени платежеспособности клиента на основании предоставленных данных. К примеру, клиенты имеющие большое количество детей более ответственно относятся к выплатам долгов, а вот клиенты с большим уровнем дохода, наоборот. Также исследование позволило сделать вывод, что стереотип в отношении низкой платежеспособности клиентов, которые берут кредит с целью проведения свадьбы, несостоятелен.

Различные данные клиента позволяют делать обоснованные прогнозы платёжоспобосноти. В развитие этой темы, в дальнейшем можно назначать подкатегории клиентов, учитывающие совокупность данных, на чем собственно говоря и основаны методики расчета скоринга.